# Spam Classifier (Naive Bayes)

In [11]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# reading through all emails
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n': # skip straight to message
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

#creating database
data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('emails/ham', 'ham'))


In [12]:
data.head()

,message,class
emails/spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
emails/spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
emails/spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


Now we will use a CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we've got a trained spam filter ready to go!

In [44]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values) #converting individual words into values

classifier = MultinomialNB() 
targets = data['class'].values
classifier.fit(counts, targets) #counting how many times certain words occur

MultinomialNB()

Let's try it out:

In [45]:
#Test examples
examples = ["Hi Bob, how about a game of golf tomorrow?", '50% discount', 'Hello John!!!', 'You have won a free phone!','Win 1 million dollars today!!']
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['ham', 'spam', 'ham', 'ham', 'spam'], dtype='<U4')

Conclusion: Since our spam data is not enough, the model is recognizing the statements such as 'You have won a free phone!' as 'ham'

Applying train/test to this spam classifier to see how well it can predict some subset of the ham and spam emails.

In [46]:
from sklearn.model_selection import train_test_split

trainDf, testDf = train_test_split(data, test_size=0.2) #splitting test to 20% size
trainVect = CountVectorizer()
trainCounts = vectorizer.fit_transform(trainDf['message'].values)
trainTargets = trainDf['class'].values

classifier.fit(trainCounts, trainTargets)

MultinomialNB()

In [47]:
testCounts = vectorizer.transform(testDf['message'])
testPredictions = classifier.predict(testCounts)

In [48]:
correct = 0
total = 0

for index, (value1, value2) in enumerate(zip(testPredictions, testDf['class'])):
    if value1 == value2:
        correct += 1
    total += 1

In [49]:
percentCorrect = correct/total

print(f'Percent correct: {percentCorrect*100}%')

Percent correct: 97.0%


In [50]:
# ALTERNATE SOLUTION

from sklearn.model_selection import train_test_split
 
train, test = train_test_split(data, test_size=0.2)
 
# Train
counts = vectorizer.fit_transform(train['message'].values)
targets = train['class'].values
classifier.fit(counts, targets)
 
# Test
counts = vectorizer.transform(test['message'].values)
predictions = classifier.predict(counts)
sum(predictions == test['class'])/(len(predictions)/100)

96.16666666666667